In [3]:
#!/usr/bin/env python3
"""
PANDA Dataset Distribution Analysis
"""
import pandas as pd

TRAIN_PATH = "/projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/data/splits/train_split.csv"
VAL_PATH = "/projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/data/splits/val_split.csv"

train_df = pd.read_csv(TRAIN_PATH)
val_df = pd.read_csv(VAL_PATH)

def map_3class(isup):
    if isup == 0: return 0
    elif isup == 1: return 1
    else: return 2

train_df['class_3'] = train_df['isup_grade'].apply(map_3class)
val_df['class_3'] = val_df['isup_grade'].apply(map_3class)

print("=" * 70)
print("PANDA DATASET DISTRIBUTION ANALYSIS")
print("=" * 70)

print(f"\nTotal: {len(train_df) + len(val_df)} WSIs (Train: {len(train_df)}, Val: {len(val_df)})")

# ISUP Distribution
print("\n" + "-" * 70)
print("ISUP GRADE DISTRIBUTION")
print("-" * 70)
print(f"\n{'ISUP':<6} {'Train':<8} {'Val':<8} {'Total':<8} {'%':<8} {'Description'}")
print("-" * 70)

descriptions = {
    0: "No cancer (Negative)",
    1: "Benign / Low-grade (Gleason 3+3)*",
    2: "Intermediate (Gleason 3+4)",
    3: "Intermediate (Gleason 4+3)",
    4: "High-grade (Gleason 4+4)",
    5: "High-grade (Gleason 4+5/5+4/5+5)"
}

total = len(train_df) + len(val_df)
for grade in range(6):
    t = len(train_df[train_df['isup_grade'] == grade])
    v = len(val_df[val_df['isup_grade'] == grade])
    pct = (t + v) / total * 100
    print(f"{grade:<6} {t:<8} {v:<8} {t+v:<8} {pct:>5.1f}%   {descriptions[grade]}")

print("\n*Note: ISUP 1 may represent low-grade cancerous tissue but is grouped")
print(" as 'Benign' due to its indolent nature and favorable prognosis.")

# 3-Class Distribution
print("\n" + "-" * 70)
print("3-CLASS MAPPING")
print("-" * 70)
print(f"\n{'Class':<12} {'ISUP':<10} {'Train':<8} {'Val':<8} {'Total':<8} {'%'}")
print("-" * 60)

class_info = {
    0: ("Background", "0"),
    1: ("Benign", "1"),
    2: ("Cancerous", "2-5")
}

for cls in range(3):
    name, isup = class_info[cls]
    t = len(train_df[train_df['class_3'] == cls])
    v = len(val_df[val_df['class_3'] == cls])
    pct = (t + v) / total * 100
    print(f"{cls} {name:<10} {isup:<10} {t:<8} {v:<8} {t+v:<8} {pct:>5.1f}%")

# Provider Analysis
print("\n" + "-" * 70)
print("DATA PROVIDER DISTRIBUTION")
print("-" * 70)

for provider in ['karolinska', 'radboud']:
    t = len(train_df[train_df['data_provider'] == provider])
    v = len(val_df[val_df['data_provider'] == provider])
    pct = (t + v) / total * 100
    print(f"\n{provider.upper()}: {t+v} WSIs ({pct:.1f}%) - Train: {t}, Val: {v}")
    
    # ISUP breakdown per provider
    print(f"  ISUP breakdown: ", end="")
    combined = pd.concat([train_df, val_df])
    provider_data = combined[combined['data_provider'] == provider]
    isup_counts = provider_data['isup_grade'].value_counts().sort_index()
    parts = [f"ISUP {g}: {isup_counts.get(g, 0)}" for g in range(6)]
    print(", ".join(parts))

# Class Imbalance
print("\n" + "-" * 70)
print("CLASS IMBALANCE")
print("-" * 70)

train_counts = train_df['class_3'].value_counts().sort_index()
ratio = train_counts.max() / train_counts.min()
print(f"\nImbalance ratio: {ratio:.2f}:1 (largest class / smallest class)")

# Train/Val Consistency
print("\n" + "-" * 70)
print("TRAIN/VAL SPLIT CONSISTENCY")
print("-" * 70)
print(f"\n{'Class':<15} {'Train %':<10} {'Val %':<10} {'Diff'}")
print("-" * 45)

for cls in range(3):
    t_pct = len(train_df[train_df['class_3'] == cls]) / len(train_df) * 100
    v_pct = len(val_df[val_df['class_3'] == cls]) / len(val_df) * 100
    diff = abs(t_pct - v_pct)
    status = "✓" if diff < 3 else "⚠"
    print(f"{cls} {class_info[cls][0]:<12} {t_pct:>5.1f}%     {v_pct:>5.1f}%     {diff:>4.1f}% {status}")

print("\n" + "=" * 70)

PANDA DATASET DISTRIBUTION ANALYSIS

Total: 10616 WSIs (Train: 8492, Val: 2124)

----------------------------------------------------------------------
ISUP GRADE DISTRIBUTION
----------------------------------------------------------------------

ISUP   Train    Val      Total    %        Description
----------------------------------------------------------------------
0      2323     569      2892      27.2%   No cancer (Negative)
1      2180     486      2666      25.1%   Benign / Low-grade (Gleason 3+3)*
2      1074     269      1343      12.7%   Intermediate (Gleason 3+4)
3      976      266      1242      11.7%   Intermediate (Gleason 4+3)
4      997      252      1249      11.8%   High-grade (Gleason 4+4)
5      942      282      1224      11.5%   High-grade (Gleason 4+5/5+4/5+5)

*Note: ISUP 1 may represent low-grade cancerous tissue but is grouped
 as 'Benign' due to its indolent nature and favorable prognosis.

----------------------------------------------------------------

In [5]:
#!/usr/bin/env python3
"""
Split validation set into validation and test sets
Final split: 80% Train, 10% Val, 10% Test
"""
import pandas as pd
from sklearn.model_selection import train_test_split

TRAIN_PATH = "/projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/data/splits/train_split.csv"
VAL_PATH = "/projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/data/splits/val_split.csv"
OUTPUT_DIR = "/projectnb/ec500kb/projects/Project_1_Team_1/Official_GTP_PANDAS/PANDAS/data/splits/"

# Load data
train_df = pd.read_csv(TRAIN_PATH)
val_df = pd.read_csv(VAL_PATH)

print("=" * 60)
print("SPLITTING VALIDATION INTO VAL + TEST")
print("=" * 60)

print(f"\nBefore split:")
print(f"  Train: {len(train_df)} ({len(train_df)/(len(train_df)+len(val_df))*100:.1f}%)")
print(f"  Val:   {len(val_df)} ({len(val_df)/(len(train_df)+len(val_df))*100:.1f}%)")

# Split validation 50/50 into new_val and test, stratified by isup_grade
new_val_df, test_df = train_test_split(
    val_df, 
    test_size=0.5, 
    random_state=42, 
    stratify=val_df['isup_grade']
)

total = len(train_df) + len(new_val_df) + len(test_df)

print(f"\nAfter split:")
print(f"  Train: {len(train_df)} ({len(train_df)/total*100:.1f}%)")
print(f"  Val:   {len(new_val_df)} ({len(new_val_df)/total*100:.1f}%)")
print(f"  Test:  {len(test_df)} ({len(test_df)/total*100:.1f}%)")

# Verify stratification
print("\n" + "-" * 60)
print("STRATIFICATION CHECK (ISUP Distribution)")
print("-" * 60)
print(f"\n{'ISUP':<6} {'Train %':<10} {'Val %':<10} {'Test %':<10}")
print("-" * 40)

for grade in range(6):
    t_pct = len(train_df[train_df['isup_grade'] == grade]) / len(train_df) * 100
    v_pct = len(new_val_df[new_val_df['isup_grade'] == grade]) / len(new_val_df) * 100
    te_pct = len(test_df[test_df['isup_grade'] == grade]) / len(test_df) * 100
    print(f"{grade:<6} {t_pct:>6.1f}%    {v_pct:>6.1f}%    {te_pct:>6.1f}%")

# 3-class check
def map_3class(isup):
    if isup == 0: return 0
    elif isup == 1: return 1
    else: return 2

train_df['class_3'] = train_df['isup_grade'].apply(map_3class)
new_val_df['class_3'] = new_val_df['isup_grade'].apply(map_3class)
test_df['class_3'] = test_df['isup_grade'].apply(map_3class)

print("\n" + "-" * 60)
print("3-CLASS DISTRIBUTION")
print("-" * 60)
class_names = {0: "Background", 1: "Benign", 2: "Cancerous"}
print(f"\n{'Class':<12} {'Train':<10} {'Val':<10} {'Test':<10}")
print("-" * 45)

for cls in range(3):
    t = len(train_df[train_df['class_3'] == cls])
    v = len(new_val_df[new_val_df['class_3'] == cls])
    te = len(test_df[test_df['class_3'] == cls])
    print(f"{cls} {class_names[cls]:<10} {t:<10} {v:<10} {te:<10}")

# Save new splits
new_val_df.drop(columns=['class_3'], inplace=True)
test_df.drop(columns=['class_3'], inplace=True)

new_val_df.to_csv(OUTPUT_DIR + "val_split_new.csv", index=False)
test_df.to_csv(OUTPUT_DIR + "test_split.csv", index=False)

print("\n" + "-" * 60)
print("FILES SAVED")
print("-" * 60)
print(f"  {OUTPUT_DIR}val_split_new.csv  ({len(new_val_df)} samples)")
print(f"  {OUTPUT_DIR}test_split.csv     ({len(test_df)} samples)")

print("\n" + "=" * 60)
print("SUMMARY")
print("=" * 60)
print(f"""
Final split:
  Train: {len(train_df):,} WSIs (80%)  -> train_split.csv (unchanged)
  Val:   {len(new_val_df):,} WSIs (10%)  -> val_split_new.csv
  Test:  {len(test_df):,} WSIs (10%)  -> test_split.csv

Total: {total:,} WSIs
""")

SPLITTING VALIDATION INTO VAL + TEST

Before split:
  Train: 8492 (80.0%)
  Val:   2124 (20.0%)

After split:
  Train: 8492 (80.0%)
  Val:   1062 (10.0%)
  Test:  1062 (10.0%)

------------------------------------------------------------
STRATIFICATION CHECK (ISUP Distribution)
------------------------------------------------------------

ISUP   Train %    Val %      Test %    
----------------------------------------
0        27.4%      26.7%      26.8%
1        25.7%      22.9%      22.9%
2        12.6%      12.7%      12.6%
3        11.5%      12.5%      12.5%
4        11.7%      11.9%      11.9%
5        11.1%      13.3%      13.3%

------------------------------------------------------------
3-CLASS DISTRIBUTION
------------------------------------------------------------

Class        Train      Val        Test      
---------------------------------------------
0 Background 2323       284        285       
1 Benign     2180       243        243       
2 Cancerous  3989       535

## Results

As shown, we split the data into 2020